In [33]:
import json, os , numpy , cv2

In [34]:
with open('./via_project.json')as file:
    json_data = json.load(file)
# json_data

In [35]:
filter_ = ['filename','x','y','width','height','cat','dog']

In [36]:
def get_images():
    name_img = os.listdir('./images')
    temp={}
    for i in name_img:
        temp[i]=cv2.imread(f'./images/{i}').copy()
    # print(temp.values())
    return temp
get_images()

{'1.jpeg': array([[[241, 234, 231],
         [241, 234, 231],
         [241, 234, 231],
         ...,
         [118, 126, 116],
         [118, 126, 116],
         [118, 126, 116]],
 
        [[241, 234, 231],
         [241, 234, 231],
         [241, 234, 231],
         ...,
         [118, 126, 116],
         [118, 126, 116],
         [118, 126, 116]],
 
        [[241, 234, 231],
         [241, 234, 231],
         [241, 234, 231],
         ...,
         [118, 126, 116],
         [118, 126, 116],
         [118, 126, 116]],
 
        ...,
 
        [[194, 196, 204],
         [193, 195, 203],
         [192, 194, 202],
         ...,
         [197, 200, 204],
         [196, 199, 203],
         [196, 199, 203]],
 
        [[194, 196, 204],
         [193, 195, 203],
         [192, 194, 202],
         ...,
         [197, 200, 204],
         [195, 198, 202],
         [193, 196, 200]],
 
        [[195, 197, 205],
         [194, 196, 204],
         [192, 194, 202],
         ...,
         [198, 201

In [37]:
def dictnary(json_data):
    list_=[]
    
    def value(key,v):
        if key in filter_  or v in filter_:
            list_.append(f'{key}:{v}')
    
    def loop (json_data):
        if isinstance(json_data , dict ):
            for k,v in json_data.items():
                if isinstance(v, (dict,list,tuple)):
                    loop(v)
                else:
                    value(k,v)
        elif isinstance(json_data, (list,tuple)):
            for i in json_data:
                if isinstance(i, (dict,list,tuple)):
                    loop(i)

    loop(json_data)
    return list_             
dictnary(json_data)


['filename:1.jpeg',
 'x:323',
 'y:36',
 'width:320',
 'height:479',
 'name:dog',
 'filename:2.jpeg',
 'x:0',
 'y:47',
 'width:160',
 'height:112',
 'name:dog',
 'x:163',
 'y:68',
 'width:97',
 'height:80',
 'name:cat',
 'filename:3.jpeg',
 'x:34',
 'y:1',
 'width:191',
 'height:151',
 'name:cat',
 'filename:4.jpeg',
 'x:67',
 'y:379',
 'width:823',
 'height:1356',
 'name:cat',
 'filename:5.jpeg',
 'x:12',
 'y:86',
 'width:770',
 'height:705',
 'name:dog',
 'filename:6.jpeg',
 'x:80',
 'y:48',
 'width:168',
 'height:226',
 'name:dog',
 'x:252',
 'y:137',
 'width:235',
 'height:142',
 'name:cat',
 'filename:7.jpeg',
 'x:22',
 'y:245',
 'width:608',
 'height:212',
 'name:cat',
 'x:586',
 'y:169',
 'width:301',
 'height:277',
 'name:dog']

In [38]:
def masked_images(img):
   masked_img = img.copy()
   masked_img[:,:]=0
   return masked_img

In [39]:
if not os.path.exists('./unet_annotations/'):
    os.mkdir('./unet_annotations/')
    if not os.path.exists('.unet_annotations/labels'):
        os.mkdir('./unet_annotations/labels')
    if not os.path.exists('./unet_annotations/train'):
        os.mkdir('./unet_annotations/train')

In [51]:
def mask_for_unet(filename,image,mask_image,points,color):
    x,y,w,h=[int(x) for x in points]
    cv2.rectangle(mask_image,(x,y),(x+w,y+h),color,-1)
    mask_image=cv2.resize(mask_image,(512,512),cv2.INTER_AREA)
    image=cv2.resize(image,(512,512),cv2.INTER_AREA)
    cv2.imwrite(f'./unet_annotations/labels/{filename}.jpg',mask_image)
    cv2.imwrite(f'./unet_annotations/train/{filename}.jpg',image)
    
    # cv2.imshow('',mask_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [55]:
data_for_unet = dictnary(json_data)
# print(data_for_unet)
images=get_images() 
image=''
mask_image=''
points=[]
for i in data_for_unet:
    # print(i)
    if i.split(':')[0] == 'filename':
        image=images[i.split(':')[1]]
        filename=i.split(':')[1].split('.')[0]
        mask_image=masked_images(image)
    else:
        if i.split(":")[0] in ['x','y','width','height']:
            points.append(i.split(":")[1])
            # print('in if')
        else:
            if i.split(":")[1] == 'cat':
                c=254
            elif i.split(":")[1] == 'dog':
                c = 253
            mask_for_unet(filename,image,mask_image,points,[c,c,c]) 
            points=[]
            # print(c)